Imports

In [45]:
import os
import ee
import geemap

out_dir = "../data/raw/"
os.makedirs(out_dir, exist_ok=True)

Initialize GEE

In [40]:
ee.Authenticate()
ee.Initialize(project='cs5245-final-project')

Define fire events

In [41]:
fire_events = {
    "dixie_fire": {
        # Dixie Fire AOI (Butte County, CA)
        "aoi": ee.Geometry.Polygon(
            [[[-121.6, 39.9], [-120.6, 39.9], [-120.6, 40.5], [-121.6, 40.5], [-121.6, 39.9]]]
        ),
        "start": "2021-07-01",
        "end": "2021-10-01"
    },
    "camp_fire": {
        # Camp Fire AOI (Butte County, CA)
        "aoi": ee.Geometry.Polygon(
            [[[-121.75, 39.6], [-121.3, 39.6], [-121.3, 40.1], [-121.75, 40.1], [-121.75, 39.6]]]
        ),
        "start": "2018-11-01",
        "end": "2018-12-15"
    },
    "bootleg_fire": {
        # Bootleg Fire AOI (Lake County, OR)
        "aoi": ee.Geometry.Polygon(
            [[[-121.2, 42.3], [-120.2, 42.3], [-120.2, 42.8], [-121.2, 42.8], [-121.2, 42.3]]]
        ),
        "start": "2021-07-01",
        "end": "2021-08-31"
    },
    "caldor_fire": {
        # Caldor Fire AOI (El Dorado County, CA)
        "aoi": ee.Geometry.Polygon(
            [[[-120.8, 38.6], [-120.2, 38.6], [-120.2, 39.1], [-120.8, 39.1], [-120.8, 38.6]]]
        ),
        "start": "2021-08-01",
        "end": "2021-10-01"
    },
    "east_troublesome_fire": {
        # East Troublesome Fire AOI (Grand County, CO)
        "aoi": ee.Geometry.Polygon(
            [[[-106.2, 40.0], [-105.6, 40.0], [-105.6, 40.4], [-106.2, 40.4], [-106.2, 40.0]]]
        ),
        "start": "2020-10-01",
        "end": "2020-11-15"
    },
    "carr_fire": {
        # Carr Fire AOI (Shasta County, CA)
        "aoi": ee.Geometry.Polygon(
            [[[-122.6, 40.4], [-122.0, 40.4], [-122.0, 40.8], [-122.6, 40.8], [-122.6, 40.4]]]
        ),
        "start": "2018-07-23",
        "end": "2018-09-04"
    },
    "thomas_fire": {
        # Thomas Fire AOI (Ventura/Santa Barbara, CA)
        "aoi": ee.Geometry.Polygon(
            [[[-120.0, 34.3], [-118.8, 34.3], [-118.8, 34.7], [-120.0, 34.7], [-120.0, 34.3]]]
        ),
        "start": "2017-12-04",
        "end": "2018-01-12"
    },
    "creek_fire": {
        # Creek Fire AOI (Fresno/Madera County, CA)
        "aoi": ee.Geometry.Polygon(
            [[[-120.6, 37.1], [-119.3, 37.1], [-119.3, 37.7], [-120.6, 37.7], [-120.6, 37.1]]]
        ),
        "start": "2020-09-04",
        "end": "2020-12-24"
    },
    "woolsey_fire": {
        # Woolsey Fire AOI (Los Angeles County, CA)
        "aoi": ee.Geometry.Polygon(
            [[[-118.9, 34.0], [-118.5, 34.0], [-118.5, 34.3], [-118.9, 34.3], [-118.9, 34.0]]]
        ),
        "start": "2018-11-08",
        "end": "2018-11-21"
    },
    "glass_fire": {
        # Glass Fire AOI (Napa/Sonoma County, CA)
        "aoi": ee.Geometry.Polygon(
            [[[-122.6, 38.4], [-122.3, 38.4], [-122.3, 38.7], [-122.6, 38.7], [-122.6, 38.4]]]
        ),
        "start": "2020-09-27",
        "end": "2020-10-20"
    }
}

In [48]:
for fire_name, fire_info in fire_events.items():
    print(f"Processing {fire_name}...")
    aoi = fire_info["aoi"]
    start, end = fire_info["start"], fire_info["end"]
    fire_dir = os.path.join(out_dir, fire_name)
    os.makedirs(fire_dir, exist_ok=True)

    # ---- Datasets ---- #

    # MODIS Burned Area (label)
    modis = (
        ee.ImageCollection("MODIS/061/MCD64A1")
        .filterDate(start, end)
        .filterBounds(aoi)
        .select("BurnDate")
        .max()
    )

    # Sentinel-2 NDVI (downsample to 30 m)
    ndvi = (
        ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
        .filterDate(start, end)
        .filterBounds(aoi)
        .map(lambda img: img.normalizedDifference(['B8', 'B4']).rename('NDVI'))
        .mean()
        .resample("bilinear")
        .reproject("EPSG:4326", None, 30)
    )

    # ESA WorldCover Landcover
    landcover = ee.Image("ESA/WorldCover/v200/2021").clip(aoi)

    # ERA5-Land Temperature (1km resolution)
    era5 = (
        ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")
        .filterDate(start, end)
        .filterBounds(aoi)
        .select("temperature_2m")
        .mean()
        .resample("bilinear")
        .reproject("EPSG:4326", None, 1000)
    )

    # CHIRPS Precipitation (5km resolution)
    chirps = (
        ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")
        .filterDate(start, end)
        .filterBounds(aoi)
        .select("precipitation")
        .sum()
        .resample("bilinear")
        .reproject("EPSG:4326", None, 5000)
    )

    # SRTM DEM
    srtm = ee.Image("USGS/SRTMGL1_003").clip(aoi)

    # ---- Export Dictionary ---- #
    datasets = {
        "modis_burned_area": modis,
        "ndvi": ndvi,
        "esa_landcover": landcover,
        "era5_mean_temp": era5,
        "chirps_precip": chirps,
        "srtm_dem": srtm
    }

    # ---- Export Loop ---- #
    for ds_name, image in datasets.items():
        out_path = os.path.join(fire_dir, f"{ds_name}.tif")
        print(f"  Exporting {fire_name} - {ds_name}...")
        try:
            geemap.ee_export_image(
                image,
                filename=out_path,
                scale=30 if ds_name == "ndvi" else None,
                region=aoi,
                file_per_band=False
            )
        except Exception as e:
            print(f"  Error exporting {ds_name}: {e}")

Processing dixie_fire...
  Exporting dixie_fire - modis_burned_area...
Generating URL ...
Please wait ...
Data downloaded to c:\Users\John Waugh\Desktop\CS Projects\CS5245-Final-Project\data\raw\dixie_fire\modis_burned_area.tif
  Exporting dixie_fire - ndvi...
Generating URL ...
Please wait ...
Data downloaded to c:\Users\John Waugh\Desktop\CS Projects\CS5245-Final-Project\data\raw\dixie_fire\ndvi.tif
  Exporting dixie_fire - esa_landcover...
Generating URL ...
An error occurred while downloading.
Total request size (173112000 bytes) must be less than or equal to 50331648 bytes.
  Exporting dixie_fire - era5_mean_temp...
Generating URL ...
Please wait ...
Data downloaded to c:\Users\John Waugh\Desktop\CS Projects\CS5245-Final-Project\data\raw\dixie_fire\era5_mean_temp.tif
  Exporting dixie_fire - chirps_precip...
Generating URL ...
Please wait ...
Data downloaded to c:\Users\John Waugh\Desktop\CS Projects\CS5245-Final-Project\data\raw\dixie_fire\chirps_precip.tif
  Exporting dixie_fire